## Matrix multiplier

Two matrices A and B

In [13]:
from pynq import Overlay
from pynq import MMIO
from time import sleep
from pynq.lib import AxiGPIO
overlay = Overlay('/home/xilinx/pynq/overlays/design/design_1.bit')
%run ./AD5628.ipynb

CTRL_WEST = AxiGPIO(overlay.ip_dict['CTRL_WEST']).channel1
ADC_1 = AxiGPIO(overlay.ip_dict['ADC_1']).channel1
ADC_2 = AxiGPIO(overlay.ip_dict['ADC_2']).channel1
ADC_3 = AxiGPIO(overlay.ip_dict['ADC_3']).channel1
ADC_4 = AxiGPIO(overlay.ip_dict['ADC_4']).channel1
AD5628 = overlay.DA4_SPI

#### Configure the DAC

In [14]:
#Initialize
sync = AxiGPIO(overlay.ip_dict['DA4_GPIO']).channel1
sync.setdirection("out")
sync.setlength(1)
ad5628 = overlay.DA4_SPI
mask = 0xffffffff
sync.write(0x1,mask)
print("AD5628 Initialized")

configure_ad5628(ad5628,0,1)
set_reference_voltage(sync, ad5628)
clear_all_channels(sync, ad5628)

AD5628 Initialized


0

#### Configure ADC reading



In [15]:
ADC_1.setdirection("in")
ADC_1.setlength(12)
ADC_2.setdirection("in")
ADC_2.setlength(12)
ADC_3.setdirection("in")
ADC_3.setlength(12)
ADC_4.setdirection("in")
ADC_4.setlength(12)
raw = ADC_1.read()

ad_mask = 0b111111111111  # Equivalent to 255 or 0xFF
masked_value = ~raw & ad_mask

# Output in binary
bit = bin(masked_value)
bit

'0b0'

#### Configure MUX ctrl

In [16]:
CTRL_WEST.setdirection("out")
CTRL_WEST.setlength(4)

cmask = 0b1111
CTRL_WEST.write(0b1111,cmask)
sleep(3)
CTRL_WEST.write(0b0000,cmask)

#### Setup PGA SPI control

In [17]:
# Write this code

#### Matrix operations

Matrix A will be programmed as weights
Matrix B will be the input vector
<td> <img src="https://dflund.se/~getz/Uploads/Memristor/CrossbarMatrix.png" alt="Drawing" style="width: 500px;"/> </td>

##### TODO:
Mapping from values to voltages, wrap this in a function.
Perhaps find largest value and map it to 2.5V and derive from there

In [18]:
matrixA = [
    [12, 7, 3, 2],
    [4, 5, 6, 1],
    [7, 8, 9, 5],
    [9, 3, 2, 11]
]

matrixB = [
    [5, 8, 1, 6]
]

# Initialize the maximum value to the first element of the matrix
max_value = matrixA[0][0]

# Iterate over each row of matrixA
for row in matrixA:
    # Iterate over each element in the row
    for element in row:
        # Update max_value if the current element is larger
        if element > max_value:
            max_value = element
            
norm_factor = max_value/2.5

In [19]:
# Start by programming weights!
# column 0

CTRL_WEST.write(0b0001,cmask)
clear_all_channels(sync, ad5628)
for i, row in enumerate(matrixA):
    write_channel(sync,ad5628,i, row[0]/norm_factor)

# column 1
CTRL_WEST.write(0b0010,cmask)
clear_all_channels(sync, ad5628)
for i, row in enumerate(matrixA):
    write_channel(sync,ad5628,i, row[1]/norm_factor)

# column 2
CTRL_WEST.write(0b0100,cmask)
clear_all_channels(sync, ad5628)
for i, row in enumerate(matrixA):
    write_channel(sync,ad5628,i, row[2]/norm_factor)

# column 3
CTRL_WEST.write(0b1000,cmask)
clear_all_channels(sync, ad5628)
for i, row in enumerate(matrixA):
    write_channel(sync,ad5628,i, row[3]/norm_factor)

# Now all weights are programmed, lets calculate.
CTRL_WEST.write(0b0000,cmask)
for j, element in enumerate(matrixB[0]):
    write_channel(sync,ad5628,j, element/norm_factor)
